<a href="https://colab.research.google.com/github/adarsh415/gpt/blob/main/gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-07-22 13:38:08--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2023-07-22 13:38:08 (32.7 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# read it to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
#let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# here are all unique characters that occurs in text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# create mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: takes a string, returns sequence of intgers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: takes list of integers, outputs string

print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
#Let's split dataset into train and validation set
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8 # or context window
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when context is: {context} target is: {target}")

when context is: tensor([18]) target is: 47
when context is: tensor([18, 47]) target is: 56
when context is: tensor([18, 47, 56]) target is: 57
when context is: tensor([18, 47, 56, 57]) target is: 58
when context is: tensor([18, 47, 56, 57, 58]) target is: 1
when context is: tensor([18, 47, 56, 57, 58,  1]) target is: 15
when context is: tensor([18, 47, 56, 57, 58,  1, 15]) target is: 47
when context is: tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is: 58


In [11]:
torch.manual_seed(1023)
batch_size=4 # how many independent sequences will process in parallel
block_size=8 # what is the maximum context length of predictions?

def get_batch(split):
  data = train_data if split=='train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('target')
print(yb.shape)
print(yb)

print('---------')

for b in range(batch_size): # batch dimension
  for t in range(block_size): # time dimension
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when context is: {context.tolist()} target is: {target}")


inputs
torch.Size([4, 8])
tensor([[53,  1, 40, 43,  1, 58, 39, 50],
        [ 1, 42, 53,  1, 52, 53, 58,  1],
        [47, 58,  1, 47, 57,  1, 52, 53],
        [25, 13, 30, 19, 13, 30, 17, 32]])
target
torch.Size([4, 8])
tensor([[ 1, 40, 43,  1, 58, 39, 50, 49],
        [42, 53,  1, 52, 53, 58,  1, 46],
        [58,  1, 47, 57,  1, 52, 53, 61],
        [13, 30, 19, 13, 30, 17, 32, 10]])
---------
when context is: [53] target is: 1
when context is: [53, 1] target is: 40
when context is: [53, 1, 40] target is: 43
when context is: [53, 1, 40, 43] target is: 1
when context is: [53, 1, 40, 43, 1] target is: 58
when context is: [53, 1, 40, 43, 1, 58] target is: 39
when context is: [53, 1, 40, 43, 1, 58, 39] target is: 50
when context is: [53, 1, 40, 43, 1, 58, 39, 50] target is: 49
when context is: [1] target is: 42
when context is: [1, 42] target is: 53
when context is: [1, 42, 53] target is: 1
when context is: [1, 42, 53, 1] target is: 52
when context is: [1, 42, 53, 1, 52] target is: 53
w

In [12]:
from torch import nn
import torch.nn.functional as F

torch.manual_seed(1033)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super(BigramLanguageModel, self).__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, target=None):
    # idx and targets are both (B,T) tensors of integer
    logits = self.token_embedding_table(idx) # (B,T,C) where C is vocab_size
    if target is not None:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      target = target.view(-1) # or target.view(B*T)
      loss = F.cross_entropy(logits, target)
    else:
      loss = None
    return logits, loss

  def generate(self, idx, max_new_char):
    #idx is (B, T) array of indices in current context
    for _ in range(max_new_char):
      # get the predictions
      logits, loss = self(idx)
      # Focus only on last time step
      logits = logits[:, -1, :] # becomes [B, C]
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B,C)
      # sample from distribution
      idx_next = torch.multinomial(probs, num_samples=1) #(B, 1)
      # append sampled index to running index
      idx = torch.cat((idx, idx_next), dim=1) # (b , T+1)

    return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
start_token = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(start_token, 100)[0].tolist()))

torch.Size([32, 65])
tensor(4.4506, grad_fn=<NllLossBackward0>)

aSqNPHtG:bIrSGCKAlEjzUMy CDulEnO!MxoxaLpOa
dINCSqlLBu:UMlNDE3ogWdO N' Nv gTfrosd?DpeSlHwy$wGgvcte:ES


In [13]:
# create Optimizer
optimizer = torch.optim.Adam(m.parameters(), lr = 1e-3)

In [14]:
eval_interval = 500
eval_iters = 200

@torch.no_grad()
def estimate_loss():
  out = {}
  m.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      xb, yb = get_batch(split)
      logits, loss = m(xb, yb)
      losses[k] = loss.item()
    out[split] = losses.mean()
  m.train()
  return out

In [15]:
batch_size = 32
for steps in range(10000):

  if steps % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batch('train')
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

step 0: train loss 4.6883, val loss 4.7080
step 500: train loss 4.1594, val loss 4.1860
step 1000: train loss 3.7305, val loss 3.7532
step 1500: train loss 3.3958, val loss 3.4180
step 2000: train loss 3.1303, val loss 3.1610
step 2500: train loss 2.9417, val loss 2.9720
step 3000: train loss 2.7997, val loss 2.8209
step 3500: train loss 2.7073, val loss 2.7331
step 4000: train loss 2.6404, val loss 2.6555
step 4500: train loss 2.5920, val loss 2.6148
step 5000: train loss 2.5543, val loss 2.5710
step 5500: train loss 2.5353, val loss 2.5638
step 6000: train loss 2.5132, val loss 2.5447
step 6500: train loss 2.4997, val loss 2.5304
step 7000: train loss 2.5031, val loss 2.5222
step 7500: train loss 2.4819, val loss 2.5090
step 8000: train loss 2.4856, val loss 2.4971
step 8500: train loss 2.4767, val loss 2.4995
step 9000: train loss 2.4661, val loss 2.4900
step 9500: train loss 2.4592, val loss 2.4858
2.4396233558654785


In [16]:
start_token = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(start_token, 1000)[0].tolist()))


Tharatome dist f s istepor
Ise, o jTO caches. t a ld mel tsal tift sgend s, keainvirinoulouraveataveind at arou'd mye; f polld, y hy g wom o id

Thashe bu seresstheaneady d then athalouco tituseerer:
Amursphuteraout we thee
TI aourt forme, thogakeay hin s mee inellenosh thin ter su, ENGoy tse p mame
O:
ENRT:
MEst, hey ses digie waghee.
Hand est.
I ird wondes Ithyo avithrar ckes!
QUpl. IOMEO:
HA:
He y, t y t y,'dill In on I yis BOLELLI'sthin, h.
MBushan, his h ve pr, h m,
HRavesengaves me ur'd brd o gesly pre wfonon himoule n oulll ice my coud!-f in, bry sirelmeseryofincane p qusen ces t dofe n, wat peno ard prplclor l fist d y, are.
INIIn fea h n
Pem n
SUKI pis seik, theathoces gnd ur?
PMale magrceidve cr gr ben Ringe whendito bime, wivedsoode my farre d d buche ILe thed y th ola soucelly weve TINCUCLy me;
Yonongrereng t ot w woures?
fa anthounorow the sth yYod ive, f wnthingavequn avirdiur'ms s.
the-I pereave t sowiver thu m:n S:
A SO:
u
Bur apad sa bttalBUKEd co? orwnchowjofoueer w 

# The Mathematical trick in self-attention

In [17]:
# consider the following toy example

torch.manual_seed(1033)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [18]:
# we want x[b,t] = mean_{i <= t} x[b, i]
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    prev = x[b,:t+1] # (T, C)
    xbow[b,t] = torch.mean(prev, 0) # mean over T

In [30]:
# Approve 2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B,T,C)
torch.allclose(xbow, xbow2)

True

In [33]:
# Version 3
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T,T)
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x

In [19]:
x[0]

tensor([[-0.4115,  1.2941],
        [-0.1356,  0.5413],
        [ 0.9311, -0.5061],
        [ 0.7235,  0.9506],
        [ 1.3993,  0.4271],
        [-0.9396,  0.1607],
        [-0.1828,  1.3827],
        [-1.3406, -0.2682]])

In [20]:
xbow[0]

tensor([[-0.4115,  1.2941],
        [-0.2735,  0.9177],
        [ 0.1280,  0.4431],
        [ 0.2769,  0.5700],
        [ 0.5013,  0.5414],
        [ 0.2612,  0.4780],
        [ 0.1978,  0.6072],
        [ 0.0055,  0.4978]])

In [23]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [27]:

torch.manual_seed(1033)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[8., 5.],
        [0., 7.],
        [7., 7.]])
c=
tensor([[8.0000, 5.0000],
        [4.0000, 6.0000],
        [5.0000, 6.3333]])
